## # 01. 메일 발송 모듈 제작

###### 앞으로 코드 한 줄의 의미가 이전 수업 보단 더 깊어짐..
###### 지난 시간에 만들었던 메일 발송 모듈 재 사용 실습

In [7]:
# 실습
# 구글 앱비밀번호: uirh ysyy jfia ktlr

# 참조할 라이브러리 정의
import os
from smtplib import SMTP
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart

#--------------------------------------------
# 메일 발송 함수 만들기 (함수(및 이름) 정의)
#--------------------------------------------
def sendMail(from_addr, to_addr, subject, content, files=[]):
    content_type = "plain"

    username = "29alsdud@gmail.com"
    password = "uirh ysyy jfia ktlr"    # (구글 앱 비밀번호)

    smtp = "smtp.gmail.com"
    port = 587

    msg = MIMEMultipart()

    msg['Subject'] = subject
    msg['From'] = from_addr
    msg['To'] = to_addr

    msg.attach(MIMEText(content, content_type))

    if files: # 여기서 file은 리스트임!
        for file_item in files:
            if os.path.exists(file_item):
                with open(file_item, 'rb') as f:
                    basename = os.path.basename(file_item)
                    part = MIMEApplication(f.read(), Name=basename)

                    part['Content-Disposition'] = 'attachment; filename="%s"' % basename
                    msg.attach(part)

                    print(basename, "(이)가 첨부 되었습니다.")


    mail = SMTP(smtp)
    mail.ehlo()
    mail.starttls()
    mail.login(username, password)
    mail.sendmail(from_addr, to_addr, msg.as_string())
    mail.quit()

In [9]:
#--------------------------------------------
# 테스트 코드
#--------------------------------------------
if __name__ == "__main__":

    sendMail("29alsdud@gmail.com", "29alsdud@gmail.com", "오늘 점심 메뉴 추천이욥", "아아, mike check mike check, ok")

## # 02. 메일링 리스트 구현

In [15]:
# .strip(): 줄 마다 있는 줄바꿈 /n 및 공백 제거
# 앞으로 룰: 
    #print(): 코딩 중간 점검용 
        # 일단 해당 줄 까지 코딩 후 출력 해보고
        # 출력 잘 되면 print 앞에 주석 처리 후 쭉 코딩하기

#### 메일 발송 준비

In [2]:
# 모듈 참조
from mylibrary import MyMailer
import datetime as dt

# 날짜 성분값 초기화
today = dt.datetime.now()
year = today.year
month = today.month
day = today.day
print(year, month, day)

2025 10 27


#### 보내는 사람, 메일 제목

In [3]:
# 메일 발신자 정보
fromAddr = "운영지원팀 <29alsdud@gmail.com>"

# 메일 제목을 위한 템플릿
subjectTmpl = "{name}님의 {yy}년 {mm}월 급여명세서 입니다."

#### 메일 본문 가져오기

In [4]:
with open('mail/content.txt', 'r', encoding='utf-8') as f:
    contentTmpl = f.read()       # .txt 파일은 utf-8임
    print(contentTmpl)

안녕하세요 {name}님

{yy}년도 {mm}월 급여명세서와 결산보고서 보내드립니다.

귀하의 노고에 감사드립니다.

- {yy}년 {mm}월 {dd}일 / 운영지원팀 드림


#### 수신자 목록 CSV에 대한 반복처리

In [5]:
with open('mail/mail_list.csv', 'r', encoding='utf-8') as f:
    csv = f.readlines()            # .csv 파일은 utf-8임

    # csv의 라인 수 만큼 반복
    for line in csv:

        # 한 줄 안의 내용 분리
        name, email, file1, file2 = line.strip().split(",")
        #print(name, email, file1, file2)

        toAddr = "{name} <{email}>".format(name = name, email = email)
        #print(toAddr)

        # 메일 제목
        subject = subjectTmpl.format(name = name, yy = year, mm = month)
        #print(subject)

        # 메일 내용
        content = contentTmpl.format(name = name, yy = year, mm = month, dd = day)
        #print(content)

        # 메일 보내기
        MyMailer.sendMail(fromAddr, toAddr, subject, content, [file1, file2])

document.pptx (이)가 첨부 되었습니다.
pay1.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay2.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay3.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay4.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay5.xlsx (이)가 첨부 되었습니다.


## # 03. 비동기 처리

##### 추후 작업할 crowling 작업을 위한 개념

In [6]:
# 동기 처리는 step-by-step 처리 -> 시간 겁나 소요
# -> 비동기 처리: 다 작업 한번에 start -> 제일 오래 걸리는 작업의 소요 시간이 마지막 작업시간
# -> by .ThreadPollExecutor()
    # ThreadPollExecutor 함수는 .futures() 객체의 내장함수임
    # 동시다발적인 ThreadPollExecutor 작업이 끝나기 전 까지 with문 안에서 대기함.

##### 실습: 메일 발송에 필요한 변수 초기화

In [9]:
# 이전 단원의 소스코드 재사용
today = dt.datetime.now()
year = today.year
month = today.month
day = today.day

fromAddr = "운영지원팀 <29alsdud@gmail.com>"
subjectTmpl = "{name}님의 {yy}년 {mm}월 급여명세서 입니다."

with open('mail/content.txt', 'r', encoding='utf-8') as f:
    contentTmpl = f.read()

##### 동기식 메일 발송

In [11]:
startTime = dt.datetime.now()

with open('mail/mail_list.csv', 'r', encoding='utf-8') as f:
    csv = f.readlines()

    for line in csv:
        name, email, file1, file2 = line.strip().split(",")
        toAddr = "{name} <{email}>".format(name = name, email = email)
        subject = subjectTmpl.format(name = name, yy = year, mm = month)
        content = contentTmpl.format(name = name, yy = year, mm = month, dd = day)

        MyMailer.sendMail(fromAddr, toAddr, subject, content, [file1, file2])

endTime = dt.datetime.now()
workTime = endTime - startTime
print("작업에 소요된 시간은 총 %s초 입니다." % workTime.seconds)

document.pptx (이)가 첨부 되었습니다.
pay1.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay2.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay3.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay4.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay5.xlsx (이)가 첨부 되었습니다.
작업에 소요된 시간은 총 19초 입니다.


##### 비동기식 메일 발송

In [15]:
import concurrent.futures as futures

In [16]:
# 동시 메일 발송 처리
startTime = dt.datetime.now()

with open('mail/mail_list.csv', 'r', encoding='utf-8') as f:
    csv = f.readlines()

    with futures.ThreadPoolExecutor(max_workers=10) as executor:
        for line in csv:
            name, email, file1, file2 = line.strip().split(",")
            toAddr = "{name} <{email}>".format(name = name, email = email)
            subject = subjectTmpl.format(name = name, yy = year, mm = month)
            content = contentTmpl.format(name = name, yy = year, mm = month, dd = day)

            MyMailer.sendMail(fromAddr, toAddr, subject, content, [file1, file2])
            executor.submit(MyMailer.sendMail, fromAddr, toAddr, subject, content, [file1, file2])

endTime = dt.datetime.now()
workTime = endTime - startTime
print("작업에 소요된 시간은 총 %s초 입니다." % workTime.seconds)

document.pptx (이)가 첨부 되었습니다.
pay1.xlsx (이)가 첨부 되었습니다.
document.pptxdocument.pptx (이)가 첨부 되었습니다.
 (이)가 첨부 되었습니다.
pay2.xlsx (이)가 첨부 되었습니다.
pay1.xlsx (이)가 첨부 되었습니다.
document.pptxdocument.pptx (이)가 첨부 되었습니다.
 (이)가 첨부 되었습니다.
pay2.xlsx (이)가 첨부 되었습니다.
pay3.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay3.xlsx (이)가 첨부 되었습니다.
pay4.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay5.xlsx (이)가 첨부 되었습니다.
pay4.xlsx (이)가 첨부 되었습니다.
document.pptx (이)가 첨부 되었습니다.
pay5.xlsx (이)가 첨부 되었습니다.
작업에 소요된 시간은 총 23초 입니다.
